In [1]:
# imports
import os
import time
import numpy as np
import pandas as pd
import json
import spotipy
import yt_dlp

from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# data loading
data_json = pd.read_json('song-ids.json')
data_json = pd.DataFrame(data_json)
data_json = data_json.rename(columns={0: 'song_id'})
data_json

,song_id
0,0pn7SynQPZBgNMlqPitrR6
1,1JsLA3JWc4XE2StbJcRYG6
2,23ssBTLuRpiE8hlCTEZCIk
3,6BPJCWBiFHF7ItUQlyyP6x
4,6GTVtjlDMEtnM2paZZ6VNe
...,...
24994,6Z1fY3aEN6dWMbLFSNCDYV
24995,2ljc7AncHJ7lZ7uTxALWLZ
24996,0B6g379TMw0nYGXA7y5xHz
24997,0VhkA3NAwIFfVW4NFCbAOD


In [3]:
# create a copy of the original dataframe (for appending song names)
data_json_copy = data_json
data_json_copy

,song_id
0,0pn7SynQPZBgNMlqPitrR6
1,1JsLA3JWc4XE2StbJcRYG6
2,23ssBTLuRpiE8hlCTEZCIk
3,6BPJCWBiFHF7ItUQlyyP6x
4,6GTVtjlDMEtnM2paZZ6VNe
...,...
24994,6Z1fY3aEN6dWMbLFSNCDYV
24995,2ljc7AncHJ7lZ7uTxALWLZ
24996,0B6g379TMw0nYGXA7y5xHz
24997,0VhkA3NAwIFfVW4NFCbAOD


In [4]:
# api creds
client_id = ''
client_secret = ''

In [5]:
# initializing api client
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [6]:
# song ids
song_ids = data_json['song_id'].tolist()
song_ids = song_ids[20000:]
print(song_ids)

['3AkdSFo7quCZ781KCqNK0T', '2RVQIq7bLi4Eu6xHPyQ46t', '7sT3O2PXs1o12XnCKlv204', '6M9L1sPBHbVMpEYV8hYiGU', '1RtoOw5CS2PBcVG8A783Q0', '4brSaQIT3bnnE41D042fgL', '1kvjYkYLtdmDNEU8KS07co', '3E7ZwUMJFqpsDOJzEkBrQ7', '1i7sjtRFGS9VNpkUBxoI9T', '2Z9U0wFqsxoALv3qtJKlwH', '39pxPloJhlabBokR8Ox1b9', '0Zykzg1CiSiWLgqLfuu813', '1kyVVCMwNlVep8rfXOTPU4', '1d9K6lxDNAT8AhcMQIUmBF', '7BBZGDSZbsb4Esi8YB94HT', '5D0fSkRiQCrEuBvk9CK0bT', '3HyPYjlDqYlQTUSoAbPSxq', '40wJM8vkUdTFMRHiBiSDqh', '4MXhiYIRDMGAuvZc5IFTwC', '0rUNZQuYQvOz6A6zwyT6tM', '1tP0WuLS6NEV7wgF4Yxszc', '3orEOtd8CPL8GFlpRpKuoE', '5XNpdKmlLJPUbwKQceX2tW', '6ZPcH0qapLpoVaiV3qgsmv', '19vVghIuHm3IAfN4xpRF3q', '0Uk589qFWtsF4l185vMKyQ', '44CZRkOxv7UItaAUmh8PgN', '4zEvqntUltAJ9IWLayoDrs', '7FITP8MVBwdQcbNAr0aVio', '3sTygkQFFT6qDmsWGBZ9jM', '3IYagvgADm3OEihjyAHdG0', '1SIzNPyqXPvSXUWx0xaCXS', '2JWwfSpvjgl9wpSgM8oNJo', '2jfk3m8nq5IvB35s2xIs1O', '7wW9scVNLYFLsmBb1yAUmS', '5qGIfWDRdyO67LCTqA2sEn', '25zlnOAzbVoOzIeXSvQFf1', '3ty70Q6EroVkAf3ZTzBzTf', '7tKiWD5g0L

In [7]:
# create download directory
download_dir = f"song_ids_downloads"
os.makedirs(download_dir, exist_ok=True)

In [8]:
# yt track download options
ydl_opts = {
    'outtmpl': f"{download_dir}/%(title)s.%(ext)s",
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}

In [ ]:
# getting track details for entire list and downloading from youtube
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    for song_id in song_ids:
        track_info = sp.track(song_id)
        title = track_info['name']
        artist = track_info['artists'][0]['name']
        
        # start indexing from 1
        print(f"Downloading {song_ids.index(song_id)} of {len(song_ids)}")
        print(f"Downloading: {title} - {artist}")
        query = f"{title} {artist}"

        try:
            ydl.download([f"ytsearch:{query}"])
            info_query = ydl.extract_info(f"ytsearch:{query}", download=False)

'''
            # get url of the first search result
            url = info_query['entries'][0]['webpage_url']

            # get title of the downloaded video and store in video_title variable
            info_url = ydl.extract_info(url, download=False)
            video_title = info_url.get('title')
'''

            print(f"Downloaded: {title} - {artist}")

            # save title to a new column in the dataframe
            data_json_copy.at[song_ids.index(song_id)+20000, 'song_name'] = f"{song_id}.mp3"

        except yt_dlp.utils.DownloadError as e:
            print(f"Error downloading {title} - {artist}: {e}")
            data_json_copy.at[song_ids.index(song_id)+20000, 'song_name'] = f"NaN"

        # 10s timeout to avoid getting rate limited
        time.sleep(10)

In [ ]:
# updated dataframe
data_json_copy

In [ ]:
# save the new dataframe as a csv
data_json_copy.to_csv('data_json_copy5.csv', index=False)